In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## Chargement images

In [8]:
data_dir = 'C:/Users/martin/Desktop/Moi/Efrei/Imagerie médicale/Project/data/Training/'

In [10]:
filenames_training = [name for name in os.listdir(data_dir)]
filenames_training

['glioma', 'meningioma', 'notumor', 'pituitary']

In [11]:
list_images = [[name for name in os.listdir(data_dir+cancer_type)] for cancer_type in filenames_training ]